In [1]:
library(PMCMRplus)
suppressPackageStartupMessages(library(reshape2))
suppressPackageStartupMessages(library(dplyr))

In [2]:
setwd('/home/ejam/documents/barley_stacks/preproc/norm_ect/umap')
d <- 158
TT <- 16
kernel <- 'umap'
dims <- 12
norm <- 'Normalized Size'

founders <- read.csv(paste('../results/ect_d74_T4.csv',sep=''))
founders_names_original <- sort(unique(founders$Founder))

founders_names <- founders_names_original
founders_names[5] <- 'CA Mariout'
founders_names[11] <- 'Good Delta'
founders_names[17] <- 'Maison Carree'
founders_names[24] <- 'Palmella Blue'
founders_names[28] <- 'WI Winter'
print(founders_names)

 [1] "Algerian"        "Alpha"           "Arequipa"        "Atlas"          
 [5] "CA Mariout"      "Club Mariout"    "Everest"         "Flynn"          
 [9] "Glabron"         "Golden Pheasant" "Good Delta"      "Han River"      
[13] "Hannchen"        "Horn"            "Lion"            "Lyallpur"       
[17] "Maison Carree"   "Manchuria"       "Meloy"           "Minia"          
[21] "Multan"          "Oderbrucker"     "Orel"            "Palmella Blue"  
[25] "Sandrel"         "Trebi"           "White Smyrna"    "WI Winter"      


In [3]:
source('~/documents/barley_stacks/barley/descriptor_analysis_utils.R')

In [6]:
info_type <- 'Combined'
filename <- paste('svm_results', 
                  tolower(info_type), d, TT, kernel, dims, 'founders.npy', sep='_')        
imat <- RcppCNPy::npyLoad(filename, "integer")

results <- base::array(0, dim=c(dim(imat)[2], dim(imat)[2], dim(imat)[1]/dim(imat)[2]))
for(i in 1:dim(results)[3]){
  results[,,i] <- t(imat[((i-1)*dim(results)[1]+1):(i*dim(results)[1]), ])
}

foolist <- get_confusion(results)
mixed_confusion <- foolist[[1]]
mixed_accuracy <- foolist[[2]]
mixed_acc <- vector()
for(j in 1:dim(results)[1])
    mixed_acc <- c(mixed_acc, 100*diag(results[,,j])/Matrix::colSums(results[,,j]))

info_type <- 'Topological'
filename <- paste('svm_results', 
                  tolower(info_type), d, TT, kernel, dims, 'founders.npy', sep='_')        
imat <- RcppCNPy::npyLoad(filename, "integer")

results <- base::array(0, dim=c(dim(imat)[2], dim(imat)[2], dim(imat)[1]/dim(imat)[2]))
for(i in 1:dim(results)[3]){
  results[,,i] <- t(imat[((i-1)*dim(results)[1]+1):(i*dim(results)[1]), ])
}

foolist <- get_confusion(results)
topo_confusion <- foolist[[1]]
topo_accuracy <- foolist[[2]]
topo_acc <- vector()
for(j in 1:dim(results)[1])
    topo_acc <- c(topo_acc, 100*diag(results[,,j])/Matrix::colSums(results[,,j]))

info_type <- 'Traditional'
filename <- '../results/svm_results_traditional_founders.rds'

results <- base::readRDS(filename)
foolist <- get_confusion(results)
trad_confusion <- foolist[[1]]
trad_accuracy <- foolist[[2]]
trad_acc <- vector()
for(j in 1:dim(results)[1])
    trad_acc <- c(trad_acc, 100*diag(results[,,j])/Matrix::colSums(results[,,j]))

In [7]:
accuracy <- cbind(diag(trad_confusion), diag(topo_confusion), diag(mixed_confusion))
colnames(accuracy) <- c('Traditional', 'Topological', 'Combined')
rownames(accuracy) <- founders_names
accuracy

,Traditional,Topological,Combined
Algerian,0.4831034,0.8669444,0.9130556
Alpha,0.2900000,0.7090909,0.8136364
Arequipa,0.6850000,0.6675000,0.8717857
Atlas,0.4237037,0.7766667,0.8845455
CA Mariout,0.6313158,0.8876596,0.9665957
Club Mariout,0.6288571,0.7972093,0.8762791
Everest,0.8242308,0.8281250,0.9450000
Flynn,0.2837500,0.6305263,0.8984211
Glabron,0.4195652,0.8551724,0.8906897
Golden Pheasant,0.4405556,0.6550000,0.7636364


In [8]:
accuracy <- cbind(trad_acc, topo_acc, mixed_acc)
colnames(accuracy) <- c('Traditional', 'Topological', 'Combined')
accuracy

Traditional,Topological,Combined
41.379310,88.88889,91.66667
38.888889,72.72727,86.36364
63.636364,64.28571,96.42857
33.333333,81.81818,84.84848
78.947368,89.36170,97.87234
65.714286,86.04651,93.02326
80.769231,84.37500,84.37500
25.000000,63.15789,94.73684
39.130435,86.20690,89.65517
61.111111,54.54545,72.72727


In [9]:
## check with friedman.test from R stats
stats::friedman.test(accuracy)


	Friedman rank sum test

data:  accuracy
Friedman chi-squared = 962.07, df = 2, p-value < 2.2e-16


In [14]:
## Chisquare distribution
PMCMRplus::friedmanTest(accuracy)
## check with friedman.test from R stats
#stats::friedman.test(accuracy)
## F-distribution
#PMCMRplus::friedmanTest(accuracy, dist = "FDist")
## Check with One-way repeated measure ANOVA
#rmat <- accuracy
#for (i in 1:length(accuracy[,1])) rmat[i,] <- rank(rmat[i,])
#dataf <- data.frame(
#y = y <- as.vector(rmat),
#g = g <- factor(c(col(accuracy))),
#b = b <- factor(c(row(accuracy))))
#summary(stats::aov(y ~ g + Error(b), data = dataf))

ERROR: Error in `levels<-.factor`(`*tmp*`, value = ROWNAMES): number of levels differs


In [16]:
?PMCMRplus::friedmanTest

In [11]:
PMCMR::posthoc.friedman.nemenyi.test(accuracy)

Registered S3 methods overwritten by 'PMCMR':
  method        from     
  print.PMCMR   PMCMRplus
  summary.PMCMR PMCMRplus




	Pairwise comparisons using Nemenyi multiple comparison test	
             with q approximation for unreplicated blocked data 

data:  accuracy 

            Traditional Topological
Topological 7.3e-15     -          
Combined    < 2e-16     < 2e-16    

P value adjustment method: none 

In [12]:
## Global test
stats::quade.test(accuracy)
## All-pairs comparisons
PMCMRplus::quadeAllPairsTest(accuracy, dist="TDist", p.adjust.method="bonferroni")
PMCMRplus::quadeAllPairsTest(accuracy, dist="Normal", p.adjust.method="bonferroni")


	Quade test

data:  accuracy
Quade F = 825.5, num df = 2, denom df = 1566, p-value < 2.2e-16



	Pairwise comparisons using Quade's test with TDist approximation 

data:  accuracy 

            Traditional Topological
Topological <2e-16      -          
Combined    <2e-16      <2e-16     

P value adjustment method: bonferroni 


	Pairwise comparisons using Quade's test	with standard-normal approximation 

data:  accuracy 

            Traditional Topological
Topological <2e-16      -          
Combined    <2e-16      <2e-16     

P value adjustment method: bonferroni 

In [29]:
?quade.test